# Project 3 CNN

In [2]:
!pip install ipython-autotime

%load_ext autotime

time: 157 µs (started: 2021-10-15 14:20:04 +00:00)


In [3]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from torchsummary import summary

time: 25 s (started: 2021-10-15 14:20:04 +00:00)


In [4]:
# CUDA 

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

time: 74.4 ms (started: 2021-10-15 14:20:29 +00:00)


## Cast and Normalize Pixel Values

In [5]:
normalize = transforms.Normalize(mean=[.5, .5, .5], std=[1 ,1, 1])
transform_ = transforms.Compose([transforms.ToTensor(), normalize])

time: 3.36 ms (started: 2021-10-15 14:20:29 +00:00)


In [6]:
#CIFAR-10 dataset.

train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform_,
                                             download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data/
time: 18.2 s (started: 2021-10-15 14:20:29 +00:00)


time: 747 µs (started: 2021-10-15 14:20:48 +00:00)


In [8]:
# Parameters

learning_rate = 0.001 # 0.001 or 0.1
mini_batch = 64 # 64 or 256
epochs = 300

time: 2.46 ms (started: 2021-10-15 14:20:48 +00:00)


In [9]:
# Data loader 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,   
                                           batch_size=mini_batch,
                                           shuffle=True) # True of False

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                          train=False,
                                          download=True, 
                                          transform=transforms.ToTensor())

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=mini_batch, 
                                          shuffle=False) # True of False


Files already downloaded and verified
time: 831 ms (started: 2021-10-15 14:20:48 +00:00)


In [23]:


class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=0),
            #nn.Conv2d(3, 24, kernel_size=5, stride=1, padding=0),
            #nn.Conv2d(3, 24, kernel_size=7, stride=1, padding=0),
            #nn.BatchNorm2d(24),
            nn.LeakyReLU(),
            #nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0),
            #nn.Conv2d(24, 48, kernel_size=3, stride=1, padding=0),
            #nn.Conv2d(24, 48, kernel_size=5, stride=1, padding=0),
            #nn.BatchNorm2d(48),
            nn.LeakyReLU(),
            #nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=0),
            #nn.Conv2d(48, 96, kernel_size=5, stride=1, padding=0),
            #nn.BatchNorm2d(96),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),

            nn.Linear(2*2*256, 512),
            nn.Dropout(p=0.3),
            nn.LeakyReLU(),
            #nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512,128),
            nn.LeakyReLU(),
            #nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, num_classes),

            nn.Softmax())
        
    def forward(self, x):
        out = self.cnn(x)

        return out

time: 28.6 ms (started: 2021-10-15 17:24:06 +00:00)


In [24]:
classes = 10

model = ConvNet(classes).to(device)
summary(model, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]           4,864
         LeakyReLU-2           [-1, 64, 28, 28]               0
       BatchNorm2d-3           [-1, 64, 28, 28]             128
         MaxPool2d-4           [-1, 64, 14, 14]               0
            Conv2d-5          [-1, 128, 12, 12]          73,856
         LeakyReLU-6          [-1, 128, 12, 12]               0
       BatchNorm2d-7          [-1, 128, 12, 12]             256
         MaxPool2d-8            [-1, 128, 6, 6]               0
            Conv2d-9            [-1, 256, 4, 4]         295,168
        MaxPool2d-10            [-1, 256, 2, 2]               0
          Flatten-11                 [-1, 1024]               0
           Linear-12                  [-1, 512]         524,800
          Dropout-13                  [-1, 512]               0
        LeakyReLU-14                  [

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


## Stochastic Gradient Descent

In [25]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9) #if needed momentum=0.9

time: 2.55 ms (started: 2021-10-15 17:24:17 +00:00)


In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Reset before!!
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/300], Step [100/782], Loss: 2.2114
Epoch [1/300], Step [200/782], Loss: 2.1865
Epoch [1/300], Step [300/782], Loss: 2.1490
Epoch [1/300], Step [400/782], Loss: 2.1165
Epoch [1/300], Step [500/782], Loss: 2.0680
Epoch [1/300], Step [600/782], Loss: 2.0646
Epoch [1/300], Step [700/782], Loss: 2.0538
Epoch [2/300], Step [100/782], Loss: 2.0506
Epoch [2/300], Step [200/782], Loss: 1.9390
Epoch [2/300], Step [300/782], Loss: 2.0063
Epoch [2/300], Step [400/782], Loss: 2.0134
Epoch [2/300], Step [500/782], Loss: 2.0052
Epoch [2/300], Step [600/782], Loss: 1.9494
Epoch [2/300], Step [700/782], Loss: 1.8860
Epoch [3/300], Step [100/782], Loss: 1.9025
Epoch [3/300], Step [200/782], Loss: 1.9171
Epoch [3/300], Step [300/782], Loss: 1.8193
Epoch [3/300], Step [400/782], Loss: 1.9158
Epoch [3/300], Step [500/782], Loss: 1.9544
Epoch [3/300], Step [600/782], Loss: 1.9883
Epoch [3/300], Step [700/782], Loss: 1.9687
Epoch [4/300], Step [100/782], Loss: 1.7922
Epoch [4/300], Step [200/782], L

In [19]:
# Test the model

model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Test Accuracy of the model on the 10000 test images: 40.24 %
time: 2.26 s (started: 2021-10-15 17:19:18 +00:00)


In [22]:
correct = 0
total = 0
with torch.no_grad():
    for (inputs, labels) in test_loader:
        images = inputs.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Accuracy of the network on the 10000 test images: 40 %
time: 2.2 s (started: 2021-10-15 17:20:46 +00:00)
